In [1]:
import pandas as pd
import numpy as np

In [8]:
users = pd.read_csv('takehome_users.csv',encoding='latin-1')
engagement = pd.read_csv('takehome_user_engagement.csv')

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0
...,...,...,...,...,...,...,...,...,...,...
11995,11996,2013-09-06 06:14:15,Meier Sophia,SophiaMeier@gustr.com,ORG_INVITE,1.378448e+09,0,0,89,8263.0
11996,11997,2013-01-10 18:28:37,Fisher Amelie,AmelieFisher@gmail.com,SIGNUP_GOOGLE_AUTH,1.358275e+09,0,0,200,NaN
11997,11998,2014-04-27 12:45:16,Haynes Jake,JakeHaynes@cuvox.de,GUEST_INVITE,1.398603e+09,1,1,83,8074.0
11998,11999,2012-05-31 11:55:59,Faber Annett,mhaerzxp@iuxiw.com,PERSONAL_PROJECTS,1.338638e+09,0,0,6,NaN


In [9]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12000 entries, 0 to 11999
Data columns (total 10 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   object_id                   12000 non-null  int64  
 1   creation_time               12000 non-null  object 
 2   name                        12000 non-null  object 
 3   email                       12000 non-null  object 
 4   creation_source             12000 non-null  object 
 5   last_session_creation_time  8823 non-null   float64
 6   opted_in_to_mailing_list    12000 non-null  int64  
 7   enabled_for_marketing_drip  12000 non-null  int64  
 8   org_id                      12000 non-null  int64  
 9   invited_by_user_id          6417 non-null   float64
dtypes: float64(2), int64(4), object(4)
memory usage: 937.6+ KB


In [10]:
engagement.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 207917 entries, 0 to 207916
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   time_stamp  207917 non-null  object
 1   user_id     207917 non-null  int64 
 2   visited     207917 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.8+ MB


In [11]:


# Convert the 'time_stamp' column to datetime format
engagement['time_stamp'] = pd.to_datetime(engagement['time_stamp'])

# Sort the values by 'user_id' and 'time_stamp'
engagement = engagement.sort_values(['user_id', 'time_stamp'])

# Resample the data to daily frequency for each user
engagement_daily = engagement.groupby(['user_id', pd.Grouper(key='time_stamp', freq='D')]).sum().reset_index()

# For each user, check for any 7-day period with 3 or more logins
engagement_daily['adopted'] = engagement_daily.groupby('user_id')['visited'].rolling(window=7, min_periods=1).sum().reset_index(level=0, drop=True) >= 3

# Get the list of adopted user ids
adopted_users = engagement_daily.loc[engagement_daily['adopted'], 'user_id'].unique()

# Add the 'adopted_user' column to the 'users' dataframe
users['adopted_user'] = users['object_id'].apply(lambda x: 1 if x in adopted_users else 0)


In [12]:
users.head()

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,adopted_user
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0,0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0,1
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0,0
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0,0
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0,0


In [13]:
# Merging the two dataframes
master_df = pd.merge(users, engagement, left_on='object_id', right_on='user_id', how='outer')


In [14]:
master_df.head()

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,adopted_user,time_stamp,user_id,visited
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0,0,2014-04-22 03:53:30,1.0,1.0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0,1,2013-11-15 03:45:04,2.0,1.0
2,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0,1,2013-11-29 03:45:04,2.0,1.0
3,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0,1,2013-12-09 03:45:04,2.0,1.0
4,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0,1,2013-12-25 03:45:04,2.0,1.0


In [15]:
# Preprocess categorical variables
master_df = pd.get_dummies(master_df, columns=['creation_source'])

# Fill missing values
# If the missing values in 'last_session_creation_time' and 'invited_by_user_id' are meaningful (i.e., 
# a missing 'last_session_creation_time' means the user has not had a session yet, a missing 
# 'invited_by_user_id' means the user was not invited by anyone), it could make sense to fill them with 0. 
# If they're not meaningful, it would be better to use a different imputation strategy.

master_df['last_session_creation_time'] = master_df['last_session_creation_time'].fillna(0)
master_df['invited_by_user_id'] = master_df['invited_by_user_id'].fillna(0)


In [22]:
from sklearn.model_selection import train_test_split

# Define the features and the target
X = master_df.drop(['adopted_user', 'object_id', 'name', 'email', 'creation_time'], axis=1)
y = master_df['adopted_user']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [27]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 168875 entries, 12787 to 121958
Data columns (total 13 columns):
 #   Column                              Non-Null Count   Dtype         
---  ------                              --------------   -----         
 0   last_session_creation_time          168875 non-null  float64       
 1   opted_in_to_mailing_list            168875 non-null  int64         
 2   enabled_for_marketing_drip          168875 non-null  int64         
 3   org_id                              168875 non-null  int64         
 4   invited_by_user_id                  168875 non-null  float64       
 5   time_stamp                          166327 non-null  datetime64[ns]
 6   user_id                             166327 non-null  float64       
 7   visited                             166327 non-null  float64       
 8   creation_source_GUEST_INVITE        168875 non-null  uint8         
 9   creation_source_ORG_INVITE          168875 non-null  uint8         
 10  crea

In [26]:
from sklearn.ensemble import RandomForestClassifier

# Initialize the model
rf = RandomForestClassifier(n_estimators=100, random_state=42)

# Fit the model
rf.fit(X_train, y_train)


TypeError: The DType <class 'numpy.dtype[datetime64]'> could not be promoted by <class 'numpy.dtype[float64]'>. This means that no common DType exists for the given inputs. For example they cannot be stored in a single array unless the dtype is `object`. The full list of DTypes is: (<class 'numpy.dtype[float64]'>, <class 'numpy.dtype[int64]'>, <class 'numpy.dtype[int64]'>, <class 'numpy.dtype[int64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[datetime64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[uint8]'>, <class 'numpy.dtype[uint8]'>, <class 'numpy.dtype[uint8]'>, <class 'numpy.dtype[uint8]'>, <class 'numpy.dtype[uint8]'>)